<a href="https://colab.research.google.com/github/kym0323/china_Portfolio/blob/main/%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90%EB%8D%B0%EC%9D%B4%ED%84%B0_ymkim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [1]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://archive.ubuntu.com/ubu

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [3]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        bs = BeautifulSoup(wd.page_source, 'lxml')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [4]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 300))
    print(get_petition(wd, 582349))
    wd.quit()

{'number': 300, 'progress': '청원종료', 'title': '대기업의 잘못된 인터넷 개인가입자 민원 처리방식과 위약금', 'like': 0, 'contents': '저는 SK인터넷,TV를 가입해서 사용하고 있는 개인 가입자 입니다.8/22일 휴가를 마치고 집으로 오니 인터넷,TV가 안되어서 SK 장애부서인 106번 으로전화하여 수리요청을 했습니다.상담원이 관할지역 기사님을 알아보고는 오늘은 기사님들이 스케줄이 안되어서 장애처리를 내일 해야 된다고 하였습니다.그래서 오늘 아이들 인터넷으로 숙제도해야되고 TV시청도 해야되니 오늘 늦더라도 고쳐달라고 말하니,조금뒤 기술부서 실장님이 전화를 주셔서 어떻게 해도 금일중으로는  기사님을 수배할수 없기 때문에 장애처리를 할수 없고 내일 오전10시즘에 처리 할수 있다고 말했습니다.A/S기사님들은 오전에 그날 A/S처리  스케줄을 다잡고 나가는데 그러면 소비자 들은 장애가 나면 무조건 다음날 A/S처리를 봤아야 되니까? 소비자 잘못이 아니라 SK장비때문에 장애가 났는데 이거는 SK에서 긴급 장애처리 기사를 더고용해서 당일 장애건도 그날A/S처리를 해줘야 되는거 아닙니까.그리고 SK잘못때문에 장애가 났는데 당일 처리를 못해주면 인터넷 해지를 요청하니 수십만원의 위약금을 물어야 된다고 하는데 이것은 잘못된 처방 아닙니까.자기들 장비때문에 장애가나서 사용못하고 그래서 수리요청하니 당일 처리가 힘들고,그러면 해지하려하니 위약금 내라고 하고,완전히 개인가입자는 봉입니까.그리고 처음 인터넷가입시 장애처리 문제에 대해서는 설명도 없었는데 장애가 나고나니 스케줄 잡아서 내일 A/S받든지 아님 나갈 사람이 없어방법이 없다.알아서 해라라는 식으로 답변을 듣기만 해야되나요.장애신고 하는 사람들은 지금 인터넷,TV가 안되어서 전화를 하는 사람들이지 미리 고장날것을 알고 전화해서 예약스케줄을 잡는 사람들이 아닙니다.SK인터넷 장애처리부서는 생각좀 하시고 일합시다.', 'answer': '', 'category': '기타', 's

## 사용예

In [5]:
import pandas as pd

## 청와대 국민청원: https://www1.president.go.kr/petitions

def get_p(시작번호 = 490001, 수집량 = 10000): # 100개당 1분정도 소요. Colab에서는 최대 1만개씩 수집 추천
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    for idx, i in enumerate(tqdm(range(시작번호-수집량, 시작번호))):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
%time rtn.append(list(get_petition(wd, 593681).values()))
%time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
결과 = pd.DataFrame(rtn, columns=c)
결과

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
480566 Message: 

480569 Message: 

480587 Message: 

480589 Message: 

480590 Message: 

480592 Message: 

480594 Message: 

480595 Message: 

480597 Message: 

480599 Message: 

480600 Message: 

480601 Message: 

480602 Message: 

480603 Message: 

480605 Message: 

480606 Message: 

480608 Message: 

480616 Message: 

480617 Message: 

480618 Message: 

480621 Message: 

480625 Message: 

480639 Message: 

480644 Message: 

480647 Message: 

480648 Message: 

480653 Message: 

480660 Message: 

480667 Message: 

480671 Message: 

480673 Message: 

480674 Message: 

480685 Message: 

480692 Message: 

480693 Message: 

480698 Message: 

480699 Message: 

480700 Message: 

480701 Message: 

480702 Message: 

480704 Message: 

480706 Message: 

480708 Message: 

480710 Message: 

480711 Message: 

480712 Message: 

480713 Message: 

480718 Message: 

480723 Message: 

480724 Message: 

480725 Message: 

480727 Message: 

480728 Message: 

480730 Mes

,number,progress,title,like,contents,answer,category,sday,eday,proponent
0,480001,청원종료,빠른년생 대처,16,현재 고3으로 살고있는 01년생입니다. 친구들과 1월1일에 놀 생각을 하고있는데 저...,,정치개혁,2018-12-27,2019-01-26,facebook
1,480002,청원종료,최저시급,28,프렌차이즈를 운영하고 있는 자영업자입니다. \n최저시급을 매년 큰폭으로 인상하시는데...,,일자리,2018-12-27,2019-01-26,naver
2,480003,error,,0,Message: \n,,,,,
3,480004,청원종료,출근시간 도로 공사좀 막아주세요,1,도로 공사로 인해 15분만에 갈 거리를 두시간에 거쳐 출근하고 있습니다. 물론 공사...,,교통/건축/국토,2018-12-27,2019-01-26,facebook
4,480005,청원종료,대한항공 와이프도 구속되었고 이제 김정호도 구속수사하라. 갑질죄목 cctv 전량조작...,10,이제 김정호의 국회의원 사퇴서를 받고 갑질관련 검찰 조사하고 구속하라.\n\ncct...,,정치개혁,2018-12-27,2019-01-26,naver
...,...,...,...,...,...,...,...,...,...,...
9997,489998,청원종료,★ ★ 이몽룡의 시 한구절 ★ ★,3,금준미주천인혈 (金樽美酒千人血)이요\n\n옥반가효만성고 (玉盤佳肴萬姓膏)라\n\n촉...,,인권/성평등,2019-01-08,2019-02-07,naver
9998,489999,청원종료,불법체류자 강력 단속 부탁합니다,11,"합법적 체류자들에게는 사회적 지원을 해주시고,,\n외국인 집단 거주 지역이나, 인력...",,행정,2019-01-08,2019-02-07,naver
9999,490000,error,,0,Message: \n,,,,,
10000,593681,청원진행중,택배기사 주 5일(토요휴무) 근무를 도와주세요,5945,존경하는 국민 여러분 !\n\n저는 강산이 두번씩 변할만큼 오랜시간을 택배업계에 근...,,교통/건축/국토,2020-10-29,2020-11-28,naver


In [6]:
결과.to_csv('국민청원데이터490001~500001.csv', header='true', encoding='utf-8-sig')

[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다